In [2]:
import cv2, os, glob
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, roc_curve

# 이미지 증강

In [8]:
def augment_images(input_folder, output_folder, num_aug=5):
    np.random.seed(42)
    os.makedirs(output_folder, exist_ok=True)

    for file in os.listdir(input_folder):
        if not file.lower().endswith(('.png','.jpeg','.jpg')):
            continue

        path = os.path.join(input_folder, file)
        img = cv2.imread(path)

        if img is None:
            continue

        name, ext = os.path.splitext(file)
        
        for i in range(num_aug):
            aug = img.copy()

            # 밝기 조절
            value = np.random.randint(-20,21)
            hsv = cv2.cvtColor(aug, cv2.COLOR_BGR2HSV)
            hsv[:, :, 2] = np.clip(hsv[:, :, 2].astype(int) + value, 0, 255).astype(np.uint8)
            aug = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

            # 회전
            angle = np.random.uniform(-15,15)  # 실수 값
            h, w = aug.shape[:2]
            M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
            aug = cv2.warpAffine(
                aug, M, (w, h),
                flags = cv2.INTER_LINEAR,
                borderMode = cv2.BORDER_REFLECT
            )

            # 좌우반전
            if np.random.rand() < 0.5:
                aug = cv2.flip(aug, 1)

            # 저장
            out_name = f"{name}_aug{i}{ext}"
            cv2.imwrite(os.path.join(output_folder, out_name), aug)

        print(f"증강 완료: {output_folder}에 결과 저장됨")

In [ ]:
# 정상이미지 복사
augment_images(
    input_folder="image/normal",
    output_folder="image/augmented/normal",
    num_aug=50
)

# 결함 이미지 복사
augment_images(
    input_folder="image/defect",
    output_folder="image/augmented/defect",
    num_aug=10
)

# 결함 탐지 코드

In [3]:
def detect_defect_final(image_path):
    img = cv2.imread(image_path)
    img_copy = img.copy()

    # 사과 영역 분리
    hsv_org = cv2.cvtColor(img_copy, cv2.COLOR_BGR2HSV)
    lower_red1 = np.array([0, 120, 80])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 120, 80])
    upper_red2 = np.array([180, 255, 255])
    apple_mask = cv2.inRange(hsv_org, lower_red1, upper_red1) + cv2.inRange(hsv_org, lower_red2, upper_red2)

    kernel_apple = np.ones((5,5), np.uint8)
    apple_mask = cv2.morphologyEx(apple_mask, cv2.MORPH_CLOSE, kernel_apple, iterations=2)
    apple_mask = cv2.morphologyEx(apple_mask, cv2.MORPH_OPEN, kernel_apple, iterations=1)
    contours_apple, _ = cv2.findContours(apple_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours_apple:
        largest_contour = max(contours_apple, key=cv2.contourArea)
        apple_mask_clean = np.zeros_like(apple_mask)
        cv2.drawContours(apple_mask_clean, [largest_contour], -1, 255, -1)
        apple_mask = apple_mask_clean

    # 반사광/꼭지 제거
    v_channel = hsv_org[:, :, 2]
    s_channel = hsv_org[:, :, 1]

    if np.mean(v_channel) > 100:
        h, w = apple_mask.shape
        top_mask = np.zeros_like(apple_mask)
        cv2.rectangle(top_mask, (0, 0), (w, int(h * 0.3)), 255, -1)
        apple_mask = cv2.bitwise_and(apple_mask, cv2.bitwise_not(top_mask))

    if np.mean(v_channel) > 100:  # 밝은 사과
        highlight_mask_v = cv2.inRange(v_channel, 210, 255)
        highlight_mask_s = cv2.inRange(s_channel, 0, 90)
    else:                         # 어두운 사과
        highlight_mask_v = cv2.inRange(v_channel, 230, 255)
        highlight_mask_s = cv2.inRange(s_channel, 0, 70)

    highlight_mask = cv2.bitwise_and(highlight_mask_v, highlight_mask_s)
    highlight_mask = cv2.morphologyEx(highlight_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=2)
    highlight_mask = cv2.morphologyEx(highlight_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=2)
    highlight_mask = cv2.bitwise_and(highlight_mask, apple_mask)

    
    # 색상 기반 멍 마스크 생성
    mean_val = np.mean(v_channel * (s_channel / 255))

    if mean_val < 90:         # 어두운 사과
        gamma = 1.25
        C_HIGH, C_LOW = 15, 5
    elif mean_val <= 115:     # 중간 밝기 사과
        gamma = 1.15
        C_HIGH, C_LOW = 18, 6
    else:                     # 밝은 사과
        gamma = 1.10
        C_HIGH, C_LOW = 21, 6

    # 감마 보정
    img_gamma = np.power(img_copy.astype(np.float32) / 255.0, gamma)
    img_gamma = np.clip(img_gamma * 255.0, 0, 255).astype(np.uint8)
    hsv = cv2.cvtColor(img_gamma, cv2.COLOR_BGR2HSV)

    # 강한 / 약한 bruise 구간 분리
    lower_bruise_strong = np.array([5, 40, 40])
    upper_bruise_strong = np.array([50, 255, 160])

    lower_bruise_weak = np.array([0, 25, 70])
    upper_bruise_weak = np.array([70, 210, 235])

    bruise_mask_strong = cv2.inRange(hsv, lower_bruise_strong, upper_bruise_strong)
    bruise_mask_weak   = cv2.inRange(hsv, lower_bruise_weak,   upper_bruise_weak)

    bruise_mask_strong = cv2.bitwise_and(bruise_mask_strong, apple_mask)
    bruise_mask_weak   = cv2.bitwise_and(bruise_mask_weak, apple_mask)

    kernel_bruise = np.ones((5,5), np.uint8)
    bruise_mask_strong = cv2.morphologyEx(bruise_mask_strong, cv2.MORPH_CLOSE, kernel_bruise, iterations=2)
    bruise_mask_weak   = cv2.morphologyEx(bruise_mask_weak,   cv2.MORPH_CLOSE, kernel_bruise, iterations=1)

    # Adaptive Threshold
    gray = cv2.cvtColor(img_gamma, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    gray_equalized = clahe.apply(gray)

    block = 41 if min(img.shape[:2]) > 200 else 21
    defect_mask_raw_high = cv2.adaptiveThreshold(
        gray_equalized, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV, block, C_HIGH
    )
    defect_mask_raw_low = cv2.adaptiveThreshold(
        gray_equalized, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV, block, C_LOW
    )

    defect_mask_high = cv2.bitwise_and(defect_mask_raw_high, apple_mask)
    defect_mask_low  = cv2.bitwise_and(defect_mask_raw_low,  apple_mask)

    # 경계 5% 제거
    h, w = apple_mask.shape
    border = int(min(h, w) * 0.05)
    margin_mask = np.zeros_like(apple_mask)
    margin_mask[border:h-border, border:w-border] = 255
    defect_mask_high = cv2.bitwise_and(defect_mask_high, margin_mask)
    defect_mask_low  = cv2.bitwise_and(defect_mask_low, margin_mask)
    bruise_mask_strong = cv2.bitwise_and(bruise_mask_strong, margin_mask)
    bruise_mask_weak   = cv2.bitwise_and(bruise_mask_weak, margin_mask)

    # 최종 결합
    bruise_mask_refined = cv2.bitwise_or(
        cv2.bitwise_and(bruise_mask_strong, defect_mask_high),
        cv2.bitwise_and(bruise_mask_weak,   defect_mask_low)
    )
    defect_mask = cv2.bitwise_or(defect_mask_high, bruise_mask_refined)
    defect_mask = cv2.bitwise_and(defect_mask, cv2.bitwise_not(highlight_mask))
    _, defect_mask = cv2.threshold(defect_mask, 127, 255, cv2.THRESH_BINARY)

    # Contour 분석
    contours, _ = cv2.findContours(defect_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    apple_area = max(1, cv2.countNonZero(apple_mask))
    valid_rects = []

    for cnt in contours:
        cnt_area = cv2.contourArea(cnt)
        if cnt_area < apple_area * 0.003 or cnt_area > apple_area * 0.08:
            continue
        x, y, w_box, h_box = cv2.boundingRect(cnt)
        if w_box * h_box > apple_area * 0.1:
            continue
        border = int(min(img_copy.shape[0], img_copy.shape[1]) * 0.05)
        if (x < border or y < border or
            x + w_box > img_copy.shape[1] - border or
            y + h_box > img_copy.shape[0] - border):
            continue
        aspect_ratio = float(w_box)/h_box
        if 0.15 < aspect_ratio < 3.5:
            valid_rects.append((x, y, x + w_box, y + h_box))
    
    # 판정
    total_defect_area = sum((x2 - x1) * (y2 - y1) for (x1, y1, x2, y2) in valid_rects)
    mean_defect_area = total_defect_area / max(len(valid_rects), 1)

    if (mean_defect_area > apple_area * 0.003 and total_defect_area > apple_area * 0.01 and len(valid_rects) >= 7):
        defect_found = True
    else:
        defect_found = False

    if defect_found:
        for (x1, y1, x2, y2) in valid_rects:
            cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 3)        

    # 스코어 계산 (ROC / F1-SCORE) 추가
    score_area = total_defect_area / apple_area
    score_count = len(valid_rects) / 7
    score = 0.7 * score_area + 0.3 * score_count

    return img_copy, defect_found, defect_mask, score

# 모델 평가

In [ ]:
image_paths = glob.glob("image/dataset/*.png")

# 라벨 설정
labels = []
for path in image_paths:
    filename = os.path.basename(path)
    prefix = filename.split("_")[0].lower()
    if prefix == "nomal":
        labels.append(0)
    elif prefix == "defect":
        labels.append(1)

# 스코어 설정
scores = []
for path in image_paths:
    _, _, _, score = detect_defect_final(path)
    scores.append(score)
scores = np.array(scores)
labels = np.array(labels)

# roc-auc 계산
roc_auc = roc_auc_score(labels, scores)
print(f"ROC-AUC: {roc_auc:.4f}")

# Precision-Recall & F1-score
precision, recall, thresholds = precision_recall_curve(labels, scores)
f1_scores = 2 * precision * recall / (precision + recall + 1e-8) 
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
best_f1 = f1_scores[best_idx]
print(f"Best F1-score: {best_f1:.4f} at threshold: {best_threshold:.4f}")

# ROC Curve 시각화
fpr, tpr, roc_thresh = roc_curve(labels, scores)
plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.3f})")
plt.plot([0,1], [0,1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()

# Precision-Recall Curve 시각화
plt.figure(figsize=(5,5))
plt.plot(recall, precision, label=f"Precision-Recall Curve (best F1={best_f1:.3f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.show()

# 탐지 결과 시각화

In [ ]:
image_folder = "image/augmented/defect"  # 정상이미지 / 결함이미지 나누어서 확인
for file in os.listdir(image_folder):
    if not file.lower().endswith((".png", ".jpg", ".jpeg")):
        continue
    image_path = os.path.join(image_folder, file)
    result_img, is_defect, _ = detect_defect_final(image_path)
    img_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(4,4))
    plt.imshow(img_rgb)
    plt.title(f"{file} - {'Defect' if is_defect else 'OK'}")
    plt.axis('off')
    plt.show()